# Demo of ROCKET transform

## Overview

ROCKET [1] transforms time series using random convolutional kernels (random length, weights, bias, dilation, and padding). ROCKET computes two features from the resulting feature maps: the max, and the proportion of positive values (or ppv). The transformed features are used to train a linear classifier.

[1] Dempster A, Petitjean F, Webb GI (2019) ROCKET: Exceptionally fast and accurate time series classification using random convolutional kernels. [arXiv:1910.13051](https://arxiv.org/abs/1910.13051)

***

## Contents

1. Imports
2. Univariate Time Series
3. Multivariate Time Series
4. Pipeline Example

***

## 1 Imports

Import example data, ROCKET, and a classifier (`RidgeClassifierCV` from scikit-learn), as well as NumPy and `make_pipeline` from scikit-learn.

**Note**: ROCKET compiles (via Numba) on import, which may take a few seconds.

In [1]:
!pip install numba
#!pip install --upgrade scikit-learn

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\program files\python38\python.exe -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import sklearn
from sklearn.linear_model import RidgeClassifierCV
from sklearn.pipeline import make_pipeline

from sktime.datasets import load_arrow_head  # univariate dataset
from sktime.datasets.base import load_japanese_vowels  # multivariate dataset
from sktime.transformers.series_as_features.rocket import Rocket
from sktime.datasets.base import _load_dataset
from sktime.utils.load_data import load_from_tsfile_to_dataframe
import os

## 2 Univariate Time Series

We can transform the data using ROCKET and separately fit a classifier, or we can use ROCKET together with a classifier in a pipeline (section 4, below).

### 2.1 Load the Training Data
For more details on the data set, see the [univariate time series classification notebook](https://github.com/alan-turing-institute/sktime/blob/master/examples/02_classification_univariate.ipynb).

In [3]:
DATA_PATH = os.path.join(os.path.dirname(r"Y:\DocShare\Bryan\MitosisAnalysis\\"))

print(os.path.isfile(r"Y:\DocShare\Bryan\MitosisAnalysis\MeanXMulti.ts"))

X_train, y_train = load_from_tsfile_to_dataframe(
    os.path.join(DATA_PATH, "MeanXMulti.ts")
)



True


### 2.2 Initialise ROCKET and Transform the Training Data

In [4]:
rocket = Rocket(num_kernels=10000)  # by default, ROCKET uses 10,000 kernels
rocket.fit(X_train)
X_train_transform = rocket.transform(X_train)

### 2.3 Fit a Classifier

We recommend using `RidgeClassifierCV` from scikit-learn for smaller datasets (fewer than approx. 20K training examples), and using logistic regression trained using stochastic gradient descent for larger datasets.

In [5]:
classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10), normalize=False)
classifier.fit(X_train_transform, y_train)

RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]))

### 2.4 Load and Transform the Test Data

In [6]:
X_test, y_test = load_from_tsfile_to_dataframe(
    os.path.join(DATA_PATH, "MeanXMulti.ts")
)
X_test_transform = rocket.transform(X_test)

### 2.5 Classify the Test Data

In [7]:
score1 = classifier.score(X_test_transform, y_test)
peachy = classifier.predict(X_test_transform)
print(peachy)
score2 = peachy==y_test
print(score1)
print('Matches,total num matching')
print(score2)
print(sum(score2))

br = []
bx = []
bt = 0 
fr = []
for i in range(1005):
    if score2[i] == False:
        br.append(y_test[i])
        bx.append(peachy[i])
        fr.append(i)
        bt = bt + y_test.astype(int)[i]
print('Ground truth',br)
print('My result',bx)
print('track IDs',fr)
q = y_test.astype(int)
print(sum(q))
print(len(br))
print(bt)

['0' '0' '0' ... '0' '0' '0']
1.0
Matches,total num matching
[ True  True  True ...  True  True  True]
1005
Ground truth []
My result []
track IDs []
134
0
0


In [8]:
for i in range(10,22):
    st1 = r"C:\Users\RegotLab\VirtualBox VMs\LocalAnalysisC\Yovel\210208\Cdc25b_Sensor_High-Resolution_Test\Output\Pos"
    st2 = st1 + str(i) + "Registration\\"
    print(st2)
    DATA_PATH2 = os.path.join(os.path.dirname(st2))
    X_test, y_test = load_from_tsfile_to_dataframe(
        os.path.join(DATA_PATH2, "MitosisClassifier.ts")
    )
    X_test_transform = rocket.transform(X_test)
    peachy = classifier.predict(X_test_transform)
    print(sum(peachy.astype(int)))
    saverPath = DATA_PATH2 + "/MitosisClassified.csv"
    np.savetxt(saverPath, peachy.astype('uint8'), delimiter=",")

C:\Users\RegotLab\VirtualBox VMs\LocalAnalysisC\Yovel\210208\Cdc25b_Sensor_High-Resolution_Test\Output\Pos10Registration\
114
C:\Users\RegotLab\VirtualBox VMs\LocalAnalysisC\Yovel\210208\Cdc25b_Sensor_High-Resolution_Test\Output\Pos11Registration\
299
C:\Users\RegotLab\VirtualBox VMs\LocalAnalysisC\Yovel\210208\Cdc25b_Sensor_High-Resolution_Test\Output\Pos12Registration\
265
C:\Users\RegotLab\VirtualBox VMs\LocalAnalysisC\Yovel\210208\Cdc25b_Sensor_High-Resolution_Test\Output\Pos13Registration\
114
C:\Users\RegotLab\VirtualBox VMs\LocalAnalysisC\Yovel\210208\Cdc25b_Sensor_High-Resolution_Test\Output\Pos14Registration\
31
C:\Users\RegotLab\VirtualBox VMs\LocalAnalysisC\Yovel\210208\Cdc25b_Sensor_High-Resolution_Test\Output\Pos15Registration\
162
C:\Users\RegotLab\VirtualBox VMs\LocalAnalysisC\Yovel\210208\Cdc25b_Sensor_High-Resolution_Test\Output\Pos16Registration\
154
C:\Users\RegotLab\VirtualBox VMs\LocalAnalysisC\Yovel\210208\Cdc25b_Sensor_High-Resolution_Test\Output\Pos17Registratio

***

## 3 Multivariate Time Series

We can use ROCKET in exactly the same way for multivariate time series.

### 3.1 Load the Training Data

In [21]:
#X_train, y_train = load_japanese_vowels(split="train", return_X_y=True)
X_train, y_train = load_from_tsfile_to_dataframe(
    os.path.join(DATA_PATH, "MeanDataMulti.ts"))

### 3.2 Initialise ROCKET and Transform the Training Data

In [22]:
rocket = Rocket()
rocket.fit(X_train)
X_train_transform = rocket.transform(X_train)

### 3.3 Fit a Classifier

In [23]:
classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10), normalize=True)
classifier.fit(X_train_transform, y_train)

RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]),
                  normalize=True)

### 3.4 Load and Transform the Test Data

In [24]:
#X_test, y_test = load_japanese_vowels(split="test", return_X_y=True)
X_test, y_test = load_from_tsfile_to_dataframe(
    os.path.join(DATA_PATH, "MeanDataMulti.ts")
)
X_test_transform = rocket.transform(X_test)

### 3.5 Classify the Test Data

In [25]:
classifier.score(X_test_transform, y_test)

0.9932236205227493

***

## 4 Pipeline Example

We can use ROCKET together with `RidgeClassifierCV` (or another classifier) in a pipeline.  We can then use the pipeline like a self-contained classifier, with a single call to `fit`, and without having to separately transform the data, etc.

### 4.1 Initialise the Pipeline

In [ ]:
rocket_pipeline = make_pipeline(
    Rocket(), RidgeClassifierCV(alphas=np.logspace(-3, 3, 10), normalize=True)
)

### 4.2 Load and Fit the Training Data

In [ ]:
X_train, y_train = load_arrow_head(split="train", return_X_y=True)

# it is necessary to pass y_train to the pipeline
# y_train is not used for the transform, but it is used by the classifier
rocket_pipeline.fit(X_train, y_train)

### 4.3 Load and Classify the Test Data

In [ ]:
X_test, y_test = load_arrow_head(split="test", return_X_y=True)

rocket_pipeline.score(X_test, y_test)